In [1]:

# independent variables - 
# start from 1 min averages
# stop wherever it hangs

# concat other features
# concat averages
# concat 1 min rsis
# concat 1 min macd


# dependent features
# category

# write everything to s3 location
# train.py
# sagemaker notebook results

In [2]:
# convert this into pipeline

In [3]:
# experiment with settings
# correlation analysis

In [4]:
import yaml
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import pyarrow.dataset as ds
import pyarrow.parquet as pq
warnings.filterwarnings("ignore")

In [5]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/sisyphus/AthenaInsights/notebooks/data_cleaning'

In [6]:
def load_yaml(loc):
    with open(loc) as stream:
        try:
            print(f'reading from {loc}')
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

### independent variables

In [7]:
client = boto3.client('s3')

In [8]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [9]:
all_paths_config_loc = '../../config/spy_30min_v1/features.yaml'
all_paths_config = load_yaml(all_paths_config_loc)
all_paths_config.keys()

reading from ../../config/spy_30min_v1/features.yaml


dict_keys(['dependent_variable', 'feature_store'])

In [10]:
all_paths_config['feature_store']['1min'].keys()

dict_keys(['stock_bars_1min_base', 'stock_bars_1min_base_macd', 'stock_bars_1min_base_other_features', 'stock_bars_1min_base_rsi', 'stock_bars_1min_diff', 'stock_bars_1min_diff_macd', 'stock_bars_1min_diff_other_features', 'stock_bars_1min_diff_rsi'])

In [11]:
def get_paths_and_cols(all_paths_config, dur='1min', file='stock_bars_1min_base', config_loc='feature_store', ):
    try:
        cols = eval(all_paths_config[config_loc][dur][file]['cols']) # stock_bars_1min_base_macd
    except:
        cols = all_paths_config[config_loc][dur][file]['cols']
    path = all_paths_config[config_loc][dur][file]['path']
    return cols, path

#### loading one min data

In [12]:
cols, path = get_paths_and_cols(all_paths_config, dur='1min', file='stock_bars_1min_diff', config_loc='feature_store', )
# path, len(cols)

In [13]:
one_min_df = pq.read_pandas(path,).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
one_min_df = one_min_df[one_min_df.symbol=='SPY']
one_min_df.head()
# df = ds.dataset(path).scanner().head(1).to_pandas()

,open_diff,high_diff,low_diff,close_diff,volume_diff,symbol,time_delta,open_diff_ema_2m,open_diff_sma_2m,high_diff_ema_2m,...,open_diff_ema_26m,open_diff_sma_26m,high_diff_ema_26m,high_diff_sma_26m,low_diff_ema_26m,low_diff_sma_26m,close_diff_ema_26m,close_diff_sma_26m,volume_diff_ema_26m,volume_diff_sma_26m
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,NaN,NaN,NaN,NaN,NaN,SPY,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 04:01:00,0.09,-0.02,0.29,-0.02,-14091.0,SPY,NaN,0.090000,NaN,-0.020000,...,0.090000,NaN,-0.020000,NaN,0.290000,NaN,-0.020000,NaN,-14091.000000,NaN
2024-01-02 04:02:00,-0.05,-0.05,-0.01,-0.01,-217.0,SPY,NaN,-0.003333,0.020,-0.040000,...,0.079630,NaN,-0.022222,NaN,0.267778,NaN,-0.019259,NaN,-13063.296296,NaN
2024-01-02 04:03:00,-0.02,-0.02,-0.01,-0.01,-5783.0,SPY,NaN,-0.014444,-0.035,-0.026667,...,0.072250,NaN,-0.022058,NaN,0.247202,NaN,-0.018573,NaN,-12524.015089,NaN
2024-01-02 04:04:00,0.00,0.00,0.00,0.00,0.0,SPY,NaN,-0.004815,-0.010,-0.008889,...,0.066898,NaN,-0.020424,NaN,0.228890,NaN,-0.017198,NaN,-11596.310268,NaN


In [14]:
one_min_df.shape

(214794, 87)

#### other features

In [15]:
cols, path = get_paths_and_cols(all_paths_config, dur='1min', file='stock_bars_1min_base_other_features', config_loc='feature_store', )
# path, len(cols)

In [16]:
cols = [x for x in cols if 'ema' not in x and 'sma' not in x and x not in ['us_eastern_timestamp']]#, 'open', 'high', 'low', 'close', 'volume',]]
# len(cols), cols

In [17]:
other_features = pq.read_pandas(path, columns=cols).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
other_features.head()

,open,high,low,close,volume,open_lag_1,high_lag_1,low_lag_1,close_lag_1,open_lag_2,...,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30,max_today,min_today,max_today_session,min_today_session
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,476.25,476.36,476.00,476.31,20460.0,NaN,NaN,NaN,NaN,NaN,...,False,False,0,0,0,0,476.36,476.0,476.36,476.0
2024-01-02 04:01:00,476.34,476.34,476.29,476.29,6369.0,476.25,476.36,476.00,476.31,NaN,...,False,False,0,0,1,1,476.36,476.0,476.36,476.0
2024-01-02 04:02:00,476.29,476.29,476.28,476.28,6152.0,476.34,476.34,476.29,476.29,476.25,...,False,False,0,0,2,2,476.36,476.0,476.36,476.0
2024-01-02 04:03:00,476.27,476.27,476.27,476.27,369.0,476.29,476.29,476.28,476.28,476.34,...,False,False,0,0,3,3,476.36,476.0,476.36,476.0
2024-01-02 04:04:00,476.27,476.27,476.27,476.27,369.0,476.27,476.27,476.27,476.27,476.29,...,False,False,0,0,4,4,476.36,476.0,476.36,476.0


In [18]:
one_min_df.shape[0]==other_features.shape[0]

True

In [19]:
one_min_df = pd.concat([one_min_df, other_features], axis=1, )

In [20]:
one_min_df.shape, other_features.shape

((214794, 192), (214794, 105))

In [21]:
one_min_df.head()

,open_diff,high_diff,low_diff,close_diff,volume_diff,symbol,time_delta,open_diff_ema_2m,open_diff_sma_2m,high_diff_ema_2m,...,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30,max_today,min_today,max_today_session,min_today_session
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,NaN,NaN,NaN,NaN,NaN,SPY,NaN,NaN,NaN,NaN,...,False,False,0,0,0,0,476.36,476.0,476.36,476.0
2024-01-02 04:01:00,0.09,-0.02,0.29,-0.02,-14091.0,SPY,NaN,0.090000,NaN,-0.020000,...,False,False,0,0,1,1,476.36,476.0,476.36,476.0
2024-01-02 04:02:00,-0.05,-0.05,-0.01,-0.01,-217.0,SPY,NaN,-0.003333,0.020,-0.040000,...,False,False,0,0,2,2,476.36,476.0,476.36,476.0
2024-01-02 04:03:00,-0.02,-0.02,-0.01,-0.01,-5783.0,SPY,NaN,-0.014444,-0.035,-0.026667,...,False,False,0,0,3,3,476.36,476.0,476.36,476.0
2024-01-02 04:04:00,0.00,0.00,0.00,0.00,0.0,SPY,NaN,-0.004815,-0.010,-0.008889,...,False,False,0,0,4,4,476.36,476.0,476.36,476.0


### dependent_variable

In [22]:
cols, path = get_paths_and_cols(all_paths_config, dur='1min', file='stock_bars_1min', config_loc='dependent_variable', )
path, len(cols), cols

('s3://sisyphus-general-bucket/AthenaInsights/latest_data/dependent_variable/stock_bars_1min.parquet',
 9,
 ['open',
  'high',
  'low',
  'close',
  'close_sma_5m',
  'category',
  'future_highs',
  'future_lows',
  'slopes'])

In [23]:
dependent_var_df = pq.read_pandas(path, columns=['category']).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
dependent_var_df.head()

,category
us_eastern_timestamp,
2024-01-02 04:00:00,C
2024-01-02 04:01:00,C
2024-01-02 04:02:00,C
2024-01-02 04:03:00,C
2024-01-02 04:04:00,C


In [24]:
dependent_var_df.loc[:'2024-09-30'].category.value_counts()

category
C    161282
A     10399
B      8554
Name: count, dtype: int64

In [25]:
dependent_var_df.loc['2024-10-01':].category.value_counts()

category
C    31241
A     1962
B     1356
Name: count, dtype: int64

In [26]:
one_min_df.shape[0]==dependent_var_df.shape[0]

True

### final dataset

In [27]:
one_min_df = pd.concat([one_min_df, dependent_var_df['category']], axis=1).drop(columns=['symbol'])
one_min_df.head()

,open_diff,high_diff,low_diff,close_diff,volume_diff,time_delta,open_diff_ema_2m,open_diff_sma_2m,high_diff_ema_2m,high_diff_sma_2m,...,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30,max_today,min_today,max_today_session,min_today_session,category
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,0,0,0,0,476.36,476.0,476.36,476.0,C
2024-01-02 04:01:00,0.09,-0.02,0.29,-0.02,-14091.0,NaN,0.090000,NaN,-0.020000,NaN,...,False,0,0,1,1,476.36,476.0,476.36,476.0,C
2024-01-02 04:02:00,-0.05,-0.05,-0.01,-0.01,-217.0,NaN,-0.003333,0.020,-0.040000,-0.035,...,False,0,0,2,2,476.36,476.0,476.36,476.0,C
2024-01-02 04:03:00,-0.02,-0.02,-0.01,-0.01,-5783.0,NaN,-0.014444,-0.035,-0.026667,-0.035,...,False,0,0,3,3,476.36,476.0,476.36,476.0,C
2024-01-02 04:04:00,0.00,0.00,0.00,0.00,0.0,NaN,-0.004815,-0.010,-0.008889,-0.010,...,False,0,0,4,4,476.36,476.0,476.36,476.0,C


In [28]:
# one_min_df['category'] = dependent_var_df['category']
# one_min_df = one_min_df.drop(columns=['us_eastern_timestamp', 'symbol'])
# one_min_df

In [29]:
', '.join(one_min_df.columns)

'open_diff, high_diff, low_diff, close_diff, volume_diff, time_delta, open_diff_ema_2m, open_diff_sma_2m, high_diff_ema_2m, high_diff_sma_2m, low_diff_ema_2m, low_diff_sma_2m, close_diff_ema_2m, close_diff_sma_2m, volume_diff_ema_2m, volume_diff_sma_2m, open_diff_ema_3m, open_diff_sma_3m, high_diff_ema_3m, high_diff_sma_3m, low_diff_ema_3m, low_diff_sma_3m, close_diff_ema_3m, close_diff_sma_3m, volume_diff_ema_3m, volume_diff_sma_3m, open_diff_ema_5m, open_diff_sma_5m, high_diff_ema_5m, high_diff_sma_5m, low_diff_ema_5m, low_diff_sma_5m, close_diff_ema_5m, close_diff_sma_5m, volume_diff_ema_5m, volume_diff_sma_5m, open_diff_ema_8m, open_diff_sma_8m, high_diff_ema_8m, high_diff_sma_8m, low_diff_ema_8m, low_diff_sma_8m, close_diff_ema_8m, close_diff_sma_8m, volume_diff_ema_8m, volume_diff_sma_8m, open_diff_ema_12m, open_diff_sma_12m, high_diff_ema_12m, high_diff_sma_12m, low_diff_ema_12m, low_diff_sma_12m, close_diff_ema_12m, close_diff_sma_12m, volume_diff_ema_12m, volume_diff_sma_12m, 

In [30]:
one_min_df.select_dtypes(include='object').columns

Index(['direction', 'category'], dtype='object')

In [31]:
one_min_df = pd.concat([one_min_df.drop(columns='direction'), pd.get_dummies(one_min_df['direction'], drop_first=True)], axis=1)

In [32]:
one_min_df.to_parquet('s3://sisyphus-general-bucket/AthenaInsights/latest_data/model/data/stock_bars_1min_diff.parquet' )